In [1]:
# importing libraries
from bs4 import BeautifulSoup,ResultSet
import requests
import pandas as pd
import numpy as np
import re
from validation import ProductValidator

In [2]:
def get_webpage(url):
    HEADERS = ({'Accept-Language': 'en-US, en;q=0.5'})
    #print(url)
    webpage = requests.get(url,headers=HEADERS)
    print(webpage)
    return webpage
    

In [3]:
all_links=[]
def get_pagination_links(webpage,class_name):
    global all_links
    soup = BeautifulSoup(webpage.content, "html.parser")
    next_page_link = soup.find_all("a", attrs = {'class': class_name})
    search_string="Next page"
    temp_link = [tag for tag in next_page_link if tag.select_one('span.icon__fallback-text:-soup-contains("' + search_string + '")')]
    links = ResultSet([])
    for l in temp_link:
        links.append(l)
    for link in links:
        if link:
            next_link = link.get('href')
            all_links.append(URL+next_link)
            get_pagination_links(get_webpage(URL+next_link),class_name)
        else:
            return

In [4]:
def get_links(webpage,class_name):
    try:
        soup = BeautifulSoup(webpage.content, "html.parser")
        links = soup.find_all("a", attrs={'class':class_name})
        links_list = []
        for link in links:
            item_link = link.get('href')
            if URL not in item_link:
                links_list.append(URL+item_link)
            else:
                links_list.append(item_link)
    except:
        links_list=[]
    return links_list

In [5]:
# getting the first page
URL = "https://foreignfortune.com/"
webpage = get_webpage(URL)

<Response [200]>


In [6]:
# finding product category links
category_links = get_links(webpage,"site-nav__link site-nav__link--main")

In [7]:
category_links

['https://foreignfortune.com//collections/men-unisex',
 'https://foreignfortune.com//collections/women',
 'https://foreignfortune.com//collections/kids',
 'https://foreignfortune.com//collections/coats-hats',
 'https://foreignfortune.com//collections/small-logo-embroidery-t-shirts-1',
 'https://foreignfortune.com//collections/frontpage',
 'https://foreignfortune.com//collections/foreign-accesories']

In [8]:
#getting all pages links
for category in category_links:
    all_links.append(category)
    get_pagination_links(get_webpage(category),"btn btn--secondary btn--narrow")
    

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [9]:
all_links

['https://foreignfortune.com//collections/men-unisex',
 'https://foreignfortune.com//collections/men-unisex?page=2',
 'https://foreignfortune.com//collections/women',
 'https://foreignfortune.com//collections/kids',
 'https://foreignfortune.com//collections/coats-hats',
 'https://foreignfortune.com//collections/small-logo-embroidery-t-shirts-1',
 'https://foreignfortune.com//collections/frontpage',
 'https://foreignfortune.com//collections/frontpage?page=2',
 'https://foreignfortune.com//collections/frontpage?page=3',
 'https://foreignfortune.com//collections/foreign-accesories',
 'https://foreignfortune.com//collections/foreign-accesories?page=2']

In [10]:
# finding product links
product_links=[]
for link in all_links:
    temp_links=get_links(get_webpage(link),"grid-view-item__link grid-view-item__image-container product-card__link")
    product_links.extend(temp_links)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [11]:
len(product_links)

57

In [12]:
product_links

['https://foreignfortune.com//collections/men-unisex/products/foreign-fortune-collection-joggers-1',
 'https://foreignfortune.com//collections/men-unisex/products/foreign-rovalf-outfit',
 'https://foreignfortune.com//collections/men-unisex/products/forign-luxury-tracksuits',
 'https://foreignfortune.com//collections/men-unisex/products/ff-coats-w-hats',
 'https://foreignfortune.com//collections/men-unisex/products/foreign-language-hoodie',
 'https://foreignfortune.com//collections/men-unisex/products/detroit-foreign-everybody-tees',
 'https://foreignfortune.com//collections/men-unisex/products/foreign-fortune-collection-short-sets',
 'https://foreignfortune.com//collections/men-unisex/products/long-socks-muti-color',
 'https://foreignfortune.com//collections/men-unisex/products/foreign-pattern-logo-socks',
 'https://foreignfortune.com//collections/men-unisex/products/foreign-fortune-socks-1',
 'https://foreignfortune.com//collections/men-unisex/products/long-foreign-socks',
 'https://f

In [13]:
def get_title(soup):
    try:
        title = soup.find('h1',attrs={'class': "product-single__title"}).get_text()
    except:
        title = ""
    return title

In [14]:
def get_price(soup):
    try:
        price =soup.find('span',attrs={'id': "ProductPrice-product-template"}).get_text().strip()
    except:
        price = ""
    return price

In [15]:
def get_dropdown_lists(soup):
    try:
        element = soup.find_all('div', attrs={'class':"selector-wrapper js product-form__item"})
        size,color,style=[],[],[]
        for ele in element:
            val = ele.find('label').get_text().strip()
            if val=="Size":
                if ele:
                    options = ele.find_all("option")
                    size = [option.text.strip() for option in options if option.text.strip()]
                else:
                    size=[]
            elif val=="Color":
                if ele:
                    options = ele.find_all("option")
                    color = [option.text.strip() for option in options if option.text.strip()]
                else:
                    color=[]
            else:
                if ele:
                    options = ele.find_all("option")
                    style = [option.text.strip() for option in options if option.text.strip()]
                else:
                    style=[]
    except:
        size,color,style=[],[],[]
    return size,color,style

In [16]:
def get_image(soup):
    try:
        image = soup.find('img', attrs={'class':"feature-row__image product-featured-img lazyload"}).get('src')
        image_url = 'https:'+image
    except:
        image_url=""
    return image_url

In [17]:
def get_variants_images(soup):
    try:
        image = soup.find_all('img', attrs={'class':"product-single__thumbnail-image"})
        images=[]
        for img in image:
            images.append('https:'+img.get('src'))
    except:
        images=[]
    return images

In [18]:
def get_description(soup):
    try:
        description=soup.find('div', attrs={'class': "product-single__description rte"}).text.strip()
    except:
        description=""
    return description

In [19]:
def get_category(soup):
    try:
        cat = soup.find('a', attrs={'class':"btn btn--secondary btn--has-icon-before return-link"})
        text=cat.get_text().strip()
        category=text.replace("Back to ","")
        
    except:
        category=""
    return category

In [20]:
# creating dictionary to store the details
d = {"product_id":[],"title":[],"description":[],"category":[],"url":[],
     "price":[],"image":[], "sizes":[], "colors":[],"styles":[],"variants_images":[]}

In [21]:
# iterating over each product link and extracting data
for link in product_links:
    new_webpage=get_webpage(link)
    new_soup = BeautifulSoup(new_webpage.content, "html.parser")
    d['product_id'].append(link.split("/")[-1])
    d['title'].append(get_title(new_soup))
    d['description'].append(get_description(new_soup))
    d['category'].append(get_category(new_soup))
    d['url'].append(link)
    d['price'].append(get_price(new_soup))
    d['image'].append(get_image(new_soup))
    size,color,style=get_dropdown_lists(new_soup)
    d['sizes'].append(size)
    d['colors'].append(color)
    d['styles'].append(style)
    d['variants_images'].append(get_variants_images(new_soup))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [22]:
len(d['title'])

57

In [23]:
# storing into csv
foreign_fortune = pd.DataFrame.from_dict(d)
foreign_fortune = foreign_fortune.dropna(subset=['title'])
foreign_fortune.to_csv("output/foreign_fortune_data.csv", header=True, index=False)

In [24]:
foreign_fortune.head()

,product_id,title,description,category,url,price,image,sizes,colors,styles,variants_images
0,foreign-fortune-collection-joggers-1,Foreign Fortune Collection Joggers- Adult,Our Foreign Fortune Collection Joggers are gre...,Men/Unisex,https://foreignfortune.com//collections/men-un...,$180.00,https://foreignfortune.com/cdn/shop/products/D...,"[XS, M, L, XL, 2X, 3X, 4X, 5X]","[Black, Green, Grey, Navy Blue, Red, Yellow]","[Hood, Crewneck]",[https://foreignfortune.com/cdn/shop/products/...
1,foreign-rovalf-outfit,FOREIGN ROVALF OUTFIT,,Men/Unisex,https://foreignfortune.com//collections/men-un...,$150.00,https://foreignfortune.com/cdn/shop/products/i...,"[xs, s, m, l, xl, 2x, 3x, 4x, 5x]","[black, purple, figi blue, tiffany blue, white...",[],[https://foreignfortune.com/cdn/shop/products/...
2,forign-luxury-tracksuits,Foreign Luxury Tracksuits,Foreign luxury tracksuits are unisex tracksuit...,Men/Unisex,https://foreignfortune.com//collections/men-un...,$150.00,https://foreignfortune.com/cdn/shop/products/9...,"[S, M, L, XL, 2X]","[Black/White, Purple/Gray/White, Cream/Brown/W...",[],[https://foreignfortune.com/cdn/shop/products/...
3,ff-coats-w-hats,FF Coats,Our Signature FF coats are great for the winte...,Men/Unisex,https://foreignfortune.com//collections/men-un...,$200.00,https://foreignfortune.com/cdn/shop/products/8...,"[XS, M, L, XL, 2X, 3X]","[Black, Red, Cream, Blue]",[],[https://foreignfortune.com/cdn/shop/products/...
4,foreign-language-hoodie,Foreign Language Hoodie,,Men/Unisex,https://foreignfortune.com//collections/men-un...,$80.00,https://foreignfortune.com/cdn/shop/products/7...,"[S, M, L, XL, 2X, 3X, 4X]","[Red, Black, Olive Green, Royal, Off-White]",[],[https://foreignfortune.com/cdn/shop/products/...


## Validating Data

In [2]:
df = pd.read_csv('output/foreign_fortune_data.csv',converters={'sizes':pd.eval,'colors':pd.eval,'styles':pd.eval,'variants_images':pd.eval})

In [3]:
df=df.fillna("")

In [4]:
df.head()

,product_id,title,description,category,url,price,image,sizes,colors,styles,variants_images
0,foreign-fortune-collection-joggers-1,Foreign Fortune Collection Joggers- Adult,Our Foreign Fortune Collection Joggers are gre...,Men/Unisex,https://foreignfortune.com//collections/men-un...,$180.00,https://foreignfortune.com/cdn/shop/products/D...,"[XS, M, L, XL, 2X, 3X, 4X, 5X]","[Black, Green, Grey, Navy Blue, Red, Yellow]","[Hood, Crewneck]",[https://foreignfortune.com/cdn/shop/products/...
1,foreign-rovalf-outfit,FOREIGN ROVALF OUTFIT,,Men/Unisex,https://foreignfortune.com//collections/men-un...,$150.00,https://foreignfortune.com/cdn/shop/products/i...,"[xs, s, m, l, xl, 2x, 3x, 4x, 5x]","[black, purple, figi blue, tiffany blue, white...",[],[https://foreignfortune.com/cdn/shop/products/...
2,forign-luxury-tracksuits,Foreign Luxury Tracksuits,Foreign luxury tracksuits are unisex tracksuit...,Men/Unisex,https://foreignfortune.com//collections/men-un...,$150.00,https://foreignfortune.com/cdn/shop/products/9...,"[S, M, L, XL, 2X]","[Black/White, Purple/Gray/White, Cream/Brown/W...",[],[https://foreignfortune.com/cdn/shop/products/...
3,ff-coats-w-hats,FF Coats,Our Signature FF coats are great for the winte...,Men/Unisex,https://foreignfortune.com//collections/men-un...,$200.00,https://foreignfortune.com/cdn/shop/products/8...,"[XS, M, L, XL, 2X, 3X]","[Black, Red, Cream, Blue]",[],[https://foreignfortune.com/cdn/shop/products/...
4,foreign-language-hoodie,Foreign Language Hoodie,,Men/Unisex,https://foreignfortune.com//collections/men-un...,$80.00,https://foreignfortune.com/cdn/shop/products/7...,"[S, M, L, XL, 2X, 3X, 4X]","[Red, Black, Olive Green, Royal, Off-White]",[],[https://foreignfortune.com/cdn/shop/products/...


In [5]:
data = df.to_dict(orient='records')

In [6]:
len(data)

57

In [7]:
validated_data=[]
invalid_data=[]

In [8]:
def check(col_name, is_valid, invalid_columns):
    if not is_valid:
        invalid_columns.append(col_name)
    return invalid_columns

In [9]:
for row in data:
    invalid_columns=[]
    is_valid_product_id = ProductValidator.validate_product_id(row['product_id'])
    invalid_columns=check('product_id',is_valid_product_id,invalid_columns)
    
    is_valid_title = ProductValidator.validate_title(row['title'])
    invalid_columns=check('title',is_valid_title,invalid_columns)
    #print(is_valid_title)
    
    is_valid_description = ProductValidator.validate_description(row['description'])
    invalid_columns=check('description',is_valid_description,invalid_columns)
    #print(is_valid_description)
    
    is_valid_url = ProductValidator.validate_url(row['url'])
    invalid_columns=check('url',is_valid_url,invalid_columns)
    #print(is_valid_url)
    
    is_valid_price = ProductValidator.validate_price(row['price'])
    invalid_columns=check('price',is_valid_price,invalid_columns)
    #print(is_valid_price)
    
    is_valid_image_url = ProductValidator.validate_image_url(row['image'])
    invalid_columns=check('image',is_valid_image_url,invalid_columns)
    #print(is_valid_image_url)
    
    is_valid_variants_images = ProductValidator.validate_variants_images(row['colors'],row['variants_images'])
    invalid_columns=check('colors and variants_images',is_valid_variants_images,invalid_columns)
    #print(is_valid_variants_images)
    
    is_valid_variants_images_urls = ProductValidator.validate_variants_images_urls(row['variants_images'])
    invalid_columns=check('variants_images',is_valid_variants_images_urls,invalid_columns)
    #print(is_valid_variants_images_urls)

    if len(invalid_columns)>0:
        row['invalid_columns']=invalid_columns
        invalid_data.append(row)
    else:
        validated_data.append(row)

In [10]:
validated_data_df = pd.DataFrame.from_dict(validated_data)

In [11]:
validated_data_df.shape

(21, 11)

In [12]:
invalid_data_df = pd.DataFrame.from_dict(invalid_data)

In [13]:
invalid_data_df.shape

(36, 12)

In [14]:
validated_data_df.to_csv("output/validated_foreign_fortune_data.csv", header=True, index=False)

In [15]:
invalid_data_df.to_csv("output/invalid_foreign_fortune_data.csv", header=True, index=False)

In [ ]:
""